In [1]:
from __future__ import division

In [2]:
import tensorflow as tf
import tflearn

In [3]:
import numpy as np

In [4]:
sess = tf.Session()
lr = tf.placeholder(tf.float32, shape=[])
optim = tf.train.AdamOptimizer(learning_rate=lr)
sgd_optim = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [5]:
from tqdm import tqdm

In [6]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [7]:
import BayesianNN

In [8]:
bnn = BayesianNN.BayesianFC([4,32,32,1])

In [9]:
def test_func(x):
    e = np.random.normal(0,0.01)
    return x + 0.3*np.sin(2 *np.pi * (x + e)) + 0.3*np.sin(4 * np.pi*(x + e)) + e

In [10]:
data = [(x, test_func(x)) for x in np.linspace(0,0.5,1000)]

In [11]:
data_input = list(map(lambda x: [x[0]], data))
data_input_poly = list(map(lambda x: [x[0], x[0]**2, x[0]**3, x[0]**4], data))
data_target = list(map(lambda x: [x[1]], data))

In [12]:
def normal_nn(optim):
    data_input = tf.placeholder(tf.float32, shape=[None, 1])
    data_target = tf.placeholder(tf.float32, shape=[None, 1])
    
    poly = tf.concat(1, [data_input, data_input**2, data_input**3, data_input**4])
    net = tflearn.fully_connected(poly, 32, activation="relu")
    net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
    
    data_output = tflearn.fully_connected(net, 1, activation="linear")
    
    loss = tf.reduce_mean(tf.square(data_output - data_target))
    op = optim.minimize(loss)
    
    return op, data_input, data_target, loss, data_output

In [13]:
batch_size = 1000

In [14]:
epoch_loss = []
nn_epoch_loss = []

In [15]:
epochs = 10000

In [16]:
test_set = [[x] for x in np.linspace(-1,1,1000)]
test_set_poly = [[x,x**2,x**3,x**4] for x in np.linspace(-1,1,1000)]
test_set_flat = [a[0] for a in test_set]
data_input_flat = [a[0] for a in data_input]
data_target_flat = [a[0] for a in data_target]

In [17]:
op, data_x, data_y, loss, data_output = normal_nn(optim)
sess.run(tf.global_variables_initializer())
for e in tqdm(range(2000)):
    input_batch = data_input
    target_batch = data_target

    _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch, lr:0.001})
    nn_epoch_loss.append(l)

100%|██████████| 2000/2000 [00:03<00:00, 533.61it/s]


In [18]:
nn_preds = sess.run(data_output, feed_dict={data_x: test_set})

In [19]:
op, data_x, data_y, m_scaling, loss, w_vars = bnn.update(16, optim, batch_size)
sess.run(tf.global_variables_initializer())

In [28]:
for e in tqdm(range(epochs)):
    indices = np.random.randint(low=0, high=1000, size=batch_size)
    input_batch = [data_input_poly[i] for i in indices]
    target_batch = [data_target[i] for i in indices]
#     input_batch = data_input_poly
#     target_batch = data_target
#     ms = (2**(epochs - e - 1))/( (2**epochs) - 1) 
    ms = 1
#     ms = (1000*epochs)/(batch_size*(e+1))
#     ms = (1000)/batch_size
    
#     grad_nums = [a for (a,b) in grads]
    _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch, m_scaling:ms, lr:0.0005})
    epoch_loss.append(l)   

100%|██████████| 10000/10000 [06:36<00:00, 25.24it/s]


In [29]:
# sess.run(bnn.qw_ps[0])
# grads = tf.gradients(loss, w_vars)
# sess.run(w_vars, feed_dict={data_x: data_input_poly, data_y:target_batch, m_scaling:[0.1]})

In [30]:
pred_input, pred_output = bnn.sample()
predictions = sess.run(pred_output, feed_dict={pred_input: test_set_poly})
var_trials = 200
for _ in tqdm(range(var_trials-1)):
    predictions = np.concatenate([predictions, sess.run(pred_output, feed_dict={pred_input: test_set_poly})], axis=1)

100%|██████████| 199/199 [00:00<00:00, 1082.70it/s]


In [31]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)
pred_means[3]

0.65510142

In [32]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [33]:
predictions.shape

(1000, 200)

In [34]:
p = figure(width=800, height=800, y_range=(-0.5,1.5), x_range=(-1,1))

err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.15))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")

for i in range(0, 200, 25):
    pp = list(map(lambda x: x[i], predictions))
    p.line(test_set_flat, pp, line_width=0.5, color=(255,0,255,0.2))

p.line(data_input_flat, data_target_flat, line_width=1, color="blue")

p.line(test_set_flat, nn_preds.flatten(), line_width=2, color="orange")

# Loss
# p.line(range(epochs), epoch_loss, line_width=2, color="green")

show(p)

In [35]:
loss_p = figure(width=500, height=500, y_range=(0,100))

# Loss
loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=0.5, color="green")

# loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=1, color="purple")

show(loss_p)